In [ ]:
import czifile
import numpy as np

import skimage.io
import skimage.filters
import skimage.feature
import skimage.morphology

import scipy

import bebi103

import holoviews as hv
hv.extension('bokeh')
import colorcet

import bokeh.io
bokeh.io.output_notebook()

In [ ]:
def show_two_ims(
    im_1,
    im_2,
    titles=[None, None],
    interpixel_distances=[0.65, 0.65],
    cmap=None,
):
    """Convenient function for showing two images side by side."""
    p_1 = bebi103.image.imshow(
        im_1,
        frame_height=200,
        title=titles[0],
        cmap=None,
        interpixel_distance=interpixel_distances[0],
        length_units="µm",
    )
    p_2 =  bebi103.image.imshow(
        im_2, 
        frame_height=200,
        title=titles[1],
        cmap=cmap, 
        interpixel_distance=interpixel_distances[0], 
        length_units="µm"
    )
    p_2.x_range = p_1.x_range
    p_2.y_range = p_1.y_range

    return bokeh.layouts.gridplot([p_1, p_2], ncols=2)

def dapi_sort(img):
    '''Reduces dimensions of dapi images to DAPI channel and z-stacks'''
    return img[0,0,0,0,:,:,:,0]

def image_reducer(filepath, zoom):
    '''Reduces the size of an inputed image'''
    # Read czi file
    img = czifile.imread(filepath)
    
    # Get rid of unnecessary dimensions
    img = dapi_sort(img)
    
    # Do a z-sum projection
    img = np.sum(img, 0)
    
    # Convert to float 
    img = (img.astype(float) - img.min()) / (img.max() - img.min())
    
    # Zoom if desired
    if zoom != False:
        return img[zoom]
    
    else:
        return img
    
def im_splitter(filepath,
               crop_size = 5000,
               split_size = 1000):
    '''Imports a large image and splits it into an array of sub-images'''
    img = image_reducer(filepath, np.s_[:crop_size, :crop_size])
    
    init_array = np.arange(0, crop_size + split_size, split_size)
    
    seg_array = [np.s_[init_array[i]:init_array[i+1]] for i in range(len(init_array)-1)]
    
    seg_all = []

    for i in seg_array:
        for j in seg_array:
            seg_all.append(np.s_[i,j])
            
    return [img[i] for i in seg_all] 
                                             
def zero_crossing_filter(im, thresh, selem_size):
    """
    Returns image with 1 if there is a zero crossing and 0 otherwise.

    thresh is the the minimal value of the gradient, as computed by Sobel
    filter, at crossing to count as a crossing.
    """
    # Square structuring element
    selem = skimage.morphology.disk(selem_size)

    # Do max filter and min filter
    im_max = scipy.ndimage.filters.maximum_filter(im, footprint=selem)
    im_min = scipy.ndimage.filters.minimum_filter(im, footprint=selem)

    # Compute gradients using Sobel filter
    im_grad = skimage.filters.sobel(im)

    # Return edges
    return ( (  ((im >= 0) & (im_min < 0))
              | ((im <= 0) & (im_max > 0)))
            & (im_grad >= thresh) )

In [ ]:
def seg_from_ls(im_float,
                thresh,
                selem_size,
                sigma,
                min_size):
    
    # LaPlacean of Gaussian
    im_LoG = scipy.ndimage.filters.gaussian_laplace(im_float, sigma)
    
    # Edge detection
    im_edge = zero_crossing_filter(im_LoG, thresh, selem_size)
    
    # Fill holes
    im_bw = scipy.ndimage.morphology.binary_fill_holes(im_edge)

    # Remove small objects
    im_bw = skimage.morphology.remove_small_objects(im_bw, min_size=1000)
    
    # Label individual nuclei and count them
    im_labeled, n_labels = skimage.measure.label(im_bw, background=0, return_num=True)
    print("Number of individual nuclei = ", n_labels)
    
    # Show result
    return bokeh.io.show(show_two_ims(im_float, im_labeled, titles=["original", "labeled"]))

def segmentation(filepath,
                thresh,
                selem_size,
                sigma,
                min_size,
                zoom = False):
    
    # Import image and reduce size
    im_float = image_reducer(filepath, zoom)
    
    # LaPlacean of Gaussian
    im_LoG = scipy.ndimage.filters.gaussian_laplace(im_float, sigma)
    
    # Edge detection
    im_edge = zero_crossing_filter(im_LoG, thresh, selem_size)
    
    # Fill holes
    im_bw = scipy.ndimage.morphology.binary_fill_holes(im_edge)

    # Remove small objects
    im_bw = skimage.morphology.remove_small_objects(im_bw, min_size=1000)
    
    # Label individual nuclei and count them
    im_labeled, n_labels = skimage.measure.label(im_bw, background=0, return_num=True)
    print("Number of individual nuclei = ", n_labels)
    
    # Show result
    return bokeh.io.show(show_two_ims(im_float, im_labeled, titles=["original", "labeled"]))

segmentation('20.02.15_DAPI-seg-3.czi',
            thresh = 0.000001,
            selem_size = 15,
            sigma = 15,
            min_size = 1000,
            zoom = np.s_[1500:3000, :1500])

In [ ]:
im3 = im_splitter('20.02.15_DAPI-seg-3.czi')

im_float = im3[13]

im_float.shape

In [ ]:
seg_from_ls(im_float,
            thresh = 0.000001,
            selem_size = 10,
            sigma = 20,
            min_size = 1000)

In [ ]:
thresh = 0.000001
selem_size = 10
sigma = 20
min_size = 1000

# LaPlacean of Gaussian
im_LoG = scipy.ndimage.filters.gaussian_laplace(im_float, sigma)
    
# Edge detection
im_edge = zero_crossing_filter(im_LoG, thresh, selem_size)
    
# Fill holes
im_bw = scipy.ndimage.morphology.binary_fill_holes(im_edge)

# Remove small objects
im_bw = skimage.morphology.remove_small_objects(im_bw, min_size=1000)
    
# Label individual nuclei and count them
im_labeled_0, n_labels_0 = skimage.measure.label(im_bw, background=0, return_num=True)

print("Number of individual nuclei = ", n_labels_0)
bokeh.io.show(show_two_ims(im_float, im_labeled_0, titles=["original", "labeled"], cmap = colorcet.coolwarm))

In [ ]:
thresh = 10**-6
selem_size = 2
sigma = 20
min_size = 1000

# LaPlacean of Gaussian
im_LoG = scipy.ndimage.filters.gaussian_laplace(im_float, sigma)
    
# Edge detection
im_edge = zero_crossing_filter(im_LoG, thresh, selem_size)

# Skeletonize edges
#im_edge = skimage.morphology.skeletonize(im_edge)

bokeh.io.show(show_two_ims(im_LoG, im_edge, cmap = colorcet.coolwarm, titles=["original", "labeled"]))

In [ ]:
# Fill holes
im_bw = scipy.ndimage.morphology.binary_fill_holes(im_edge)

# Remove small objects
im_bw = skimage.morphology.remove_small_objects(im_bw, min_size=2000)
    
# Label individual nuclei and count them
im_labeled_0, n_labels_0 = skimage.measure.label(im_bw, background=0, return_num=True)

print(n_labels_0)
bokeh.io.show(show_two_ims(im_float, im_labeled_0, cmap = colorcet.b_glasbey_hv, titles=["original", "labeled"]))

In [ ]:
# Filter image
im_filt_tv = skimage.restoration.denoise_tv_chambolle(im_float, 0.05)

# Show filtered image
bokeh.io.show(
    show_two_ims(im_float, im_filt_tv, titles=["original", "tv filtered"])
)


In [ ]:
thresh = 10**-5
selem_size = 10
sigma = 10
min_size = 1000

# LaPlacean of Gaussian
im_LoG = scipy.ndimage.filters.gaussian_laplace(im_float, sigma)
    
# Edge detection
im_edge = zero_crossing_filter(im_LoG, thresh, selem_size)

# Skeletonize edges
im_edge = skimage.morphology.skeletonize(im_edge)

bokeh.io.show(show_two_ims(im_LoG, im_edge, cmap = colorcet.coolwarm, titles=["original", "labeled"]))

In [ ]:
# Fill holes
im_bw = scipy.ndimage.morphology.binary_fill_holes(im_edge)

# Remove small objects
im_bw = skimage.morphology.remove_small_objects(im_bw, min_size=3000)
    
# Label individual nuclei and count them
im_labeled_0, n_labels_0 = skimage.measure.label(im_bw, background=0, return_num=True)

print(n_labels_0)
bokeh.io.show(show_two_ims(im_float, im_labeled_0, cmap = colorcet.b_glasbey_hv, titles=["original", "labeled"]))